In [1]:
!pip install transformers

In [2]:
from sklearn.ensemble import GradientBoostingRegressor
import transformers
import torch
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import LinearSVR
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
train_data = pd.read_csv('/kaggle/input/amazon-ml/dataset/train.csv')
test_data = pd.read_csv('/kaggle/input/amazon-ml/dataset/test.csv')

In [4]:
train_data

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,NaN,1650,2125.980000
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",NaN,2755,393.700000
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574
4,283658,The United Empire Loyalists: A Chronicle of th...,NaN,NaN,6112,598.424000
...,...,...,...,...,...,...
2249693,2422167,Nike Women's As W Ny Df Swsh Hn Kh Bra (CZ7610...,Material : Polyester,NaN,3009,1181.100000
2249694,2766635,"(3PCS) Goose Game Cute Cartoon Enamel Pins, Fu...",[❤ [Inspiration] Inspired by the Untitled Goos...,<p><b>[Brand]: </b>XVIEONR</p> <p><br></p> <p>...,3413,125.984252
2249695,1987786,Kangroo Sweep Movement Printed Wooden Wall Clo...,"[Dial size: 12 inches in diameter,Big, clear r...",Wall Clocks Are Very Attractive In Looks And E...,1574,1200.000000
2249696,1165754,Electro Voice EKX-BRKT15 | Wall Mount Bracket ...,NaN,NaN,592,2900.000000


In [5]:
train_data = train_data.sample(n=20000, random_state=123, replace = True)

In [6]:
train_data = train_data[train_data['PRODUCT_LENGTH'] >= 0]
train_data['PRODUCT_LENGTH'] = np.log1p(train_data['PRODUCT_LENGTH'])

In [7]:
train_set, val_set = train_test_split(train_data, test_size=0.2, random_state=123)

In [8]:
vectorizer = TfidfVectorizer()
train_features = vectorizer.fit_transform(train_set['TITLE'].fillna('') + ' ' + train_set['DESCRIPTION'].fillna('') + ' ' + train_set['BULLET_POINTS'].fillna(''))
val_features = vectorizer.transform(val_set['TITLE'].fillna('') + ' ' + val_set['DESCRIPTION'].fillna('') + ' ' + val_set['BULLET_POINTS'].fillna(''))

In [9]:
model = LinearSVR(max_iter=15000)
model.fit(train_features, train_set['PRODUCT_LENGTH'].fillna(0))

LinearSVR(max_iter=15000)

In [10]:
val_predictions = model.predict(val_features)

In [11]:
mse = mean_squared_error(val_set['PRODUCT_LENGTH'], val_predictions)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.752955019964365


In [12]:
test_features = vectorizer.transform(test_data['TITLE'].fillna('') + ' ' + test_data['DESCRIPTION'].fillna('') + ' ' + test_data['BULLET_POINTS'].fillna(''))
test_predictions = model.predict(test_features)
submission_df = pd.DataFrame({'PRODUCT_ID': test_data['PRODUCT_ID'], 'PRODUCT_LENGTH': test_predictions})

In [13]:
submission_df.to_csv("submission3.csv", index=False)